# SQL Introduction

SQL is a powerful language, which can get you extremly far. It is concise and sufficiently simple (once you get used to it). Spark supports SQL very well and is continuously improving the implementation by adding new features and providing better compatibility with other existing databases like Postgres etc.

In this notebook we will give a solid introduction to the all important `SELECT` statements, which provide similar capabilites like all Spark DataFrame transformations.

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

if not 'spark' in locals():
    spark = SparkSession.builder \
        .master("local[*]") \
        .config("spark.driver.memory","4G") \
        .getOrCreate()

spark

/opt/anaconda3/lib/python3.10/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/11 10:11:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## 1 Watson Sales Product Sample Data

First we load the data, which is provided as a single CSV file, which again is well supported by Apache Spark

In [46]:
basedir = "s3://dimajix-training/data/"
#basedir = "/home/kaya/Jupyter/Dimajix/pyspark-datascience/data/"

In [47]:
persons = spark.read.csv(basedir + "persons_header.csv", header=True, inferSchema=True)
persons.toPandas()

,age,height,name,sex
0,23,156,Alice,female
1,21,181,Bob,male
2,27,176,Charlie,male
3,24,167,Eve,female
4,19,172,Frances,female
5,31,191,George,female


In order to work with SQL, we immediately register the Spark DataFrame as a temporary view.

In [15]:
persons.createOrReplaceTempView("persons")

# 2. Simple Transformations

Following the original Spark introductionary notebook, we start with very simple transformations. These are always executed inside a `SELECT` statement. The first version uses the simplest structure:

```sql
SELECT
    expression_1 AS result_column_1,
    expression_2 AS result_column_2,
    ...
FROM some_table_name
```

In [17]:
query = """
    SELECT
        lower(name) AS name,
        upper(sex),
        height*0.3937008 AS height_inch
    FROM persons
"""

# Execute query and display the result
result = spark.sql(query)
result.toPandas()

,name,upper(sex),height_inch
0,alice,FEMALE,61.4173248
1,bob,MALE,71.2598448
2,charlie,MALE,69.2913408
3,eve,FEMALE,65.7480336
4,frances,FEMALE,67.7165376
5,george,FEMALE,75.1968528


Lets look at a different example where we want to create a new DataFrame with the appropriate salutation in front of the name. We achieve this by the following `SELECT` statement:

In [18]:
query = """
    SELECT
        concat(
            CASE 
                WHEN sex = 'female' THEN 'Mrs '
                WHEN sex = 'male' THEN 'Mr '
                ELSE 'Hello '
            END,
            name
        ) AS salutation
    FROM persons
"""

# Execute query and display the result
result = spark.sql(query)
result.toPandas()

,salutation
0,Mrs Alice
1,Mr Bob
2,Mr Charlie
3,Mrs Eve
4,Mrs Frances
5,Mrs George


## Exercise

Using the `persons` temp view, perform the following operations:
* Add a new column `status` which should be `child` if the person is younger than 18 and `adult` otherwise
* Replace the column `name` by a new column `hashed_name` containing the hash value of the name
* Drop the column `sex`

In [20]:
query = """
    SELECT
        CASE 
            WHEN age < 18 THEN 'child'
            ELSE 'adult'
        END AS status,
        hash(name) AS hased_name,
        height
    FROM persons
"""

# Execute query and display the result
result = spark.sql(query)
result.toPandas()

,status,hased_name,height
0,adult,-1597477876,156
1,adult,549595493,181
2,adult,906014075,176
3,adult,-1915804868,167
4,adult,1222495376,172
5,adult,-454693442,191


# 3. Filtering

*Filtering* denotes the process of keeping only rows which meet a certain filter criteria. SQL uses a `WHERE` condition to specify which records should be kept. Note that the conditions in the `WHERE` clause refer to the original table, not to the columns specified in the `SELECT`.

In [22]:
query = """
    SELECT
       *
    FROM persons
    WHERE age > 25
"""

# Execute query and display the result
result = spark.sql(query)
result.toPandas()

,age,height,name,sex
0,27,176,Charlie,male
1,31,191,George,female


## Exercise
Perform two different filter operations (with two different result sets):
* Select all women with a height of at least 160
* Select all persons which are younger than 20 or older than 30

In [23]:
query = """
    SELECT
       *
    FROM persons
    WHERE sex = 'female' AND height >= 160
"""

# Execute query and display the result
result = spark.sql(query)
result.toPandas()

,age,height,name,sex
0,24,167,Eve,female
1,19,172,Frances,female
2,31,191,George,female


In [24]:
query = """
    SELECT
       *
    FROM persons
    WHERE age < 20 OR age > 30
"""

# Execute query and display the result
result = spark.sql(query)
result.toPandas()

,age,height,name,sex
0,19,172,Frances,female
1,31,191,George,female


# 4. Simple Aggregations

SQL supports aggregations without grouping inside a `SELECT` statement.

In [31]:
query = """
    SELECT
       count(*) AS count,
       avg(height) AS avg_height,
       min(age) AS min_age
    FROM persons
"""

# Execute query and display the result
result = spark.sql(query)
result.toPandas()

,count,avg_height,min_age
0,6,173.833333,19


# 5. Grouping & Aggregation

An important class of operation is grouping and aggregation, which is provided in SQL via a `SELECT aggregation GROUP BY grouping` statement. Note that in contrast to PySpark, you need to explictily add the grouping column to the list of expressions in order to see it in the result.

In [35]:
query = """
    SELECT
       sex,
       avg(age) AS avg_age,
       min(age) AS min_age
    FROM persons
    GROUP BY sex
"""

# Execute query and display the result
result = spark.sql(query)
result.toPandas()

,sex,avg_age,min_age
0,female,24.25,19
1,male,24.00,21


## Exercise

Using the `persons` temp view, calculate the average height and the number of records per sex.

In [36]:
query = """
    SELECT
       sex,
       avg(height) AS avg_height,
       count(*) AS count
    FROM persons
    GROUP BY sex
"""

# Execute query and display the result
result = spark.sql(query)
result.toPandas()

,sex,avg_height,count
0,female,171.5,4
1,male,178.5,2


# 6. Sorting

SQL also supports sorting data with the `ORDER BY` clause. For example we can sort all persons by their height as follows:

In [42]:
query = """
    SELECT
       *
    FROM persons
    ORDER BY height
"""

# Execute query and display the result
result = spark.sql(query)
result.toPandas()

,age,height,name,sex
0,23,156,Alice,female
1,24,167,Eve,female
2,19,172,Frances,female
3,27,176,Charlie,male
4,21,181,Bob,male
5,31,191,George,female


If nothing else is specified, SQL will sort the records in increasing order of the sort columns. If you require descending order, this can be specified by manipulating the sort column with the `DESC` modifier as follows:

In [43]:
query = """
    SELECT
       *
    FROM persons
    ORDER BY height DESC
"""

# Execute query and display the result
result = spark.sql(query)
result.toPandas()

,age,height,name,sex
0,31,191,George,female
1,21,181,Bob,male
2,27,176,Charlie,male
3,19,172,Frances,female
4,24,167,Eve,female
5,23,156,Alice,female


## Exercise

As an exercise we want to sort all persons first by their sex and then by their descening age. Sorting by multiple columns can easily be achieved by specifying multiple columns separated with a comma in the `ORDER BY` clause.

In [44]:
query = """
    SELECT
       *
    FROM persons
    ORDER BY sex, age DESC
"""

# Execute query and display the result
result = spark.sql(query)
result.toPandas()

,age,height,name,sex
0,31,191,George,female
1,24,167,Eve,female
2,23,156,Alice,female
3,19,172,Frances,female
4,27,176,Charlie,male
5,21,181,Bob,male


# 7. Joining Data

Every relation algebra also contains join operations which lets you combine multiple tables by a matching criterion. SQL also supports joins of multiple tables/views. In order to shed some light on that, we need a second DataFrame in addition to the `persons` temp view. Therefore we load some address data as follows:

In [48]:
addresses = spark.read.json(basedir + "addresses.json")
addresses.createOrReplaceTempView("addresses")
addresses.toPandas()

,city,name
0,Hamburg,Alice
1,Frankfurt,Bob
2,Berlin,Henry


Now that we have the `addresses` view, we want to combine it with the `persons` view such that the city of every person is added as a new column. This is achieved by the `JOIN` clause which together with two parameters: The first parameter specifies the second DataFrame to join with, and the second parameter specifies the join condition. In this case we want to join all records, where the `name` column matches.

In [49]:
query = """
    SELECT
       *
    FROM persons p
    JOIN addresses a ON p.name = a.name
"""

# Execute query and display the result
result = spark.sql(query)
result.toPandas()

,age,height,name,sex,city,name
0,23,156,Alice,female,Hamburg,Alice
1,21,181,Bob,male,Frankfurt,Bob


Let me make some relevant remarks:
* The resulting DataFrame now contains two `name` columns - one comes from the `persons` view, the other from the `addresses` view. Since the join condition could have used some more complex expression, this behaviour is only logical since SQL cannot assume that all joins simply use directly some column value. For example we could also have transformed the column on the fly by converting the name to upper case directly inside the join condition.
* The result contains only persons where an address was found, although the original `persons` view contained more persons.
* There are no records of addresses without any person, although the `addresses` view contains information about some persons not available in the `persons` DataFrame.

So let us first address the first observation. We can easily get rid of the copied `name` column by either performing an explicit select of the desired columns, or by dropping the duplicate columns. The duplicate `name` columns can be addressed by their alias:

In [51]:
query = """
    SELECT
       coalesce(p.name, a.name) AS name,
       p.age,
       p.height,
       p.sex,
       a.city
    FROM persons p
    JOIN addresses a ON p.name = a.name
"""

# Execute query and display the result
result = spark.sql(query)
result.toPandas()

,name,age,height,sex,city
0,Alice,23,156,female,Hamburg
1,Bob,21,181,male,Frankfurt


Now let us explain the last two observations. These are due to the used join type, which was a so called *inner* join. In this case, only records with information from both sides of the `JOIN` are included in the result.

In addition to the *inner* join, SQL also supports some additional joins:
* *outer join* will contain records for all elements from both DataFrames. If either the left or right DataFrames doesn't contain any information, the result will contain `None` values (= `NULL` values) for the corresponding columns.
* In a *right join*, the second view (the right view) as specified as an argument is the leading element. The result will contain records for every record in that view.
* In a *left join*, the first view (the left view) as specified as the object iteself is the leading element. The result will contain records for every record in that view.

In [52]:
query = """
    SELECT
       *
    FROM persons p
    FULL OUTER JOIN addresses a ON p.name = a.name
"""

# Execute query and display the result
result = spark.sql(query)
result.toPandas()

,age,height,name,sex,city,name
0,23.0,156.0,Alice,female,Hamburg,Alice
1,21.0,181.0,Bob,male,Frankfurt,Bob
2,27.0,176.0,Charlie,male,None,None
3,24.0,167.0,Eve,female,None,None
4,19.0,172.0,Frances,female,None,None
5,31.0,191.0,George,female,None,None
6,NaN,NaN,None,None,Berlin,Henry


In [53]:
query = """
    SELECT
       *
    FROM persons p
    LEFT JOIN addresses a ON p.name = a.name
"""

# Execute query and display the result
result = spark.sql(query)
result.toPandas()

,age,height,name,sex,city,name
0,23,156,Alice,female,Hamburg,Alice
1,21,181,Bob,male,Frankfurt,Bob
2,27,176,Charlie,male,None,None
3,24,167,Eve,female,None,None
4,19,172,Frances,female,None,None
5,31,191,George,female,None,None


In [54]:
query = """
    SELECT
       *
    FROM persons p
    RIGHT JOIN addresses a ON p.name = a.name
"""

# Execute query and display the result
result = spark.sql(query)
result.toPandas()

,age,height,name,sex,city,name
0,23.0,156.0,Alice,female,Hamburg,Alice
1,21.0,181.0,Bob,male,Frankfurt,Bob
2,NaN,NaN,None,None,Berlin,Henry


# 8. Combining all Features

A general `SELECT` statement looks as follows (we omit CTEs for the moment)

```sql
SELECT
    expression_1 AS column_1,
    expression_2 AS column_2,
    ...
FROM table_1
JOIN table_2 ON ...
JOIN table_3 ON ...
WHERE ...
GROUP BY ...
HAVING ...
ORDER BY ...
LIMIT n
```

The different parts need to be specified in exactly this order. And they are also evaluated in exactly this order, which means:
* In the first step, all records are read from the table specified in the `FROM` clause.
* Then `JOIN` clauses are executed by reading the appropriate tables and matching the records. There can be multiple `JOIN` clauses.
* Then the `WHERE` clause is executed, i.e. all records are filtered.
* Then the `GROUP BY` clause is executed
* Then the `HAVING` clause is executed. It serves as an additional filter criteria *after* grouped aggregation.
* Now all column expressions in the `SELECT` part are evaulated.
* The result set is sorted according to the `ORDER BY` clause
* The first `n` records are taken accoring to the `LIMIT` clause

Of course, the SQL optimizer may execute things in a different order. But the conceptional ordering is exactly as described above.

In [59]:
query = """
    SELECT
        sex,
       COUNT(*) AS count
    FROM persons p
    LEFT JOIN addresses a ON p.name = a.name
    WHERE p.age > 23
    GROUP BY sex
    HAVING COUNT(*) > 1
    ORDER BY count
    LIMIT 3
"""

# Execute query and display the result
result = spark.sql(query)
result.toPandas()

,sex,count
0,female,2
